In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-08-01-01-01-02.wav
/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-01-01-01-01-02.wav
/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-07-02-01-02-02.wav
/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-07-01-01-02-02.wav
/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-01-01-02-01-02.wav
/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-06-02-02-01-02.wav
/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-04-01-02-01-02.wav
/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-01-01-01-02-02.wav
/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-02-01-01-02-02.wav
/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-03-01-01-01-02.wav
/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-07-01-02-02-02.wav
/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-05-01-01-02-02.wav
/kaggle/input/codealpha-speechrecognition/Actor_02/03-01-03-01-01-02-02.wav
/kaggle/inpu

In [ ]:
# ============================
# Task 2 — Emotion Recognition (RAVDESS-style dataset)
# Path: /kaggle/input/codealpha-speechrecognition
# ============================

import os
import glob
import numpy as np
import pandas as pd
import librosa
import librosa.display
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

# -------------------------
# Settings
# -------------------------
DATA_DIR = "/kaggle/input/codealpha-speechrecognition"
SAMPLE_RATE = 22050        # librosa default
DURATION = 3.0             # seconds: load fixed duration (trunc/pad)
SAMPLES_PER_TRACK = int(SAMPLE_RATE * DURATION)
N_MFCC = 40
MAX_PAD_LEN = 130          # MFCC time frames — tune if needed

# RAVDESS-style emotion code mapping (filename example: 03-01-08-01-01-01-02.wav)
# third field (index 2) is emotion code in RAVDESS
emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# -------------------------
# Utility: extract MFCC (with padding/truncation)
# -------------------------
def extract_mfcc(file_path, n_mfcc=N_MFCC, max_pad_len=MAX_PAD_LEN):
    try:
        y, sr = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION, offset=0.0)
    except Exception as e:
        # if loading fails, return None
        return None
    # ensure length: pad or truncate
    if len(y) < SAMPLES_PER_TRACK:
        pad_width = SAMPLES_PER_TRACK - len(y)
        y = np.pad(y, (0, pad_width), mode='constant')
    else:
        y = y[:SAMPLES_PER_TRACK]
    # compute MFCCs (n_mfcc x T)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    # transpose -> T x n_mfcc, we prefer shape (n_mfcc, T)
    # pad/truncate time dimension to max_pad_len
    if mfcc.shape[1] < max_pad_len:
        pad_width = max_pad_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0),(0,pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_pad_len]
    return mfcc

# -------------------------
# Walk dataset and build features + labels
# -------------------------
filepaths = glob.glob(os.path.join(DATA_DIR, "**", "*.wav"), recursive=True)
print("Found audio files:", len(filepaths))

X = []
y = []
bad_files = []

for fp in tqdm(filepaths):
    fname = os.path.basename(fp)
    parts = fname.split("-")
    # Expect at least 3 parts: e.g. 03-01-08-01-01-01-02.wav -> emotion code at index 2
    if len(parts) >= 3:
        emo_code = parts[2]
        label = emotion_map.get(emo_code)
        if label is None:
            # skip unknown emotion codes
            continue
        mfcc = extract_mfcc(fp, n_mfcc=N_MFCC, max_pad_len=MAX_PAD_LEN)
        if mfcc is None:
            bad_files.append(fp)
            continue
        X.append(mfcc)
        y.append(label)
    else:
        # fallback: skip files that don't match pattern
        bad_files.append(fp)

print("Loaded samples:", len(X))
if bad_files:
    print("Skipped files (count):", len(bad_files))

X = np.array(X)   # shape: (N, n_mfcc, max_pad_len)
y = np.array(y)

# -------------------------
# Shuffle and train-test split
# -------------------------
# Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)
n_classes = len(le.classes_)
print("Classes:", le.classes_)

# Expand dims for CNN input: (N, n_mfcc, max_pad_len, 1)
X_exp = X[..., np.newaxis]
print("X shape for model:", X_exp.shape)

X_train, X_test, y_train, y_test = train_test_split(X_exp, y_enc, test_size=0.2, random_state=42, stratify=y_enc)
y_train_cat = to_categorical(y_train, num_classes=n_classes)
y_test_cat = to_categorical(y_test, num_classes=n_classes)

print("Train shape:", X_train.shape, y_train_cat.shape)
print("Test shape:", X_test.shape, y_test_cat.shape)

# -------------------------
# Build a compact CNN model
# -------------------------
input_shape = (N_MFCC, MAX_PAD_LEN, 1)

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.3),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(n_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# -------------------------
# Train
# -------------------------
EPOCHS = 30
BATCH_SIZE = 32

history = model.fit(
    X_train, y_train_cat,
    validation_data=(X_test, y_test_cat),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=2
)

# -------------------------
# Save model
# -------------------------
OUT = "/kaggle/working"
model_path = os.path.join(OUT, "emotion_cnn.h5")
model.save(model_path)
print("Saved model to:", model_path)

# -------------------------
# Evaluate & report
# -------------------------
test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
print(f"Test accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}")

# Predictions & classification report
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
print("Classification report:\n")
print(classification_report(y_test, y_pred, target_names=le.classes_))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion matrix")
plt.colorbar()
tick_marks = np.arange(len(le.classes_))
plt.xticks(tick_marks, le.classes_, rotation=45)
plt.yticks(tick_marks, le.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()

# -------------------------
# Plot training curves
# -------------------------
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Loss')
plt.legend()
plt.show()


Found audio files: 2880


 18%|█▊        | 532/2880 [00:36<01:47, 21.82it/s] 